## 🛠️ **ETL (Extract, Transform, Load)**



En este ipynb vamos a Extraer los archivos y cargarlos en parquet. Luego en otro archivo haremos las transformaciones.

Tambien vamos a estar utilizando la funcion personalizada personalizada `data_type_check` invocada desde `data_utils.py` para dejar un vistazo rápido del dataframe y  poder observar:
- Variables categóricas
- Variables numéricas
- Dimensiones del dataframe
- Nulos
- Tipos de datos
- Informacion acerca de los datos faltantes o nulos de cada columna    


####  **Importamos las librerías que vamos a usar**


In [7]:
import gc
import os
import warnings

import pandas as pd
from data_utils import data_type_check, data_type_check_pkl

warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import seaborn as sns

___

## Dataset Yelp

### business.pkl

In [6]:
business = pd.read_pickle('../0_Dataset/Yelp/business.pkl')
business.info()
business.sample(2)

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
15195,u_0e1X9whtdIBURmiQCv_A,Burrito Boarder,17 3rd St N,Saint Petersburg,TN,33701,27.771694,-82.636936,3.0,190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99735,QTmwmIb8AHhyQBB2Q19xsg,Tommy's Express® Car Wash,1240 Missouri Ave N,Largo,DE,33770,27.928671,-82.78711,3.5,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data_type_check(business)
business.sample(2)


 Resumen del dataframe:

Dimensiones:  (150243, 14)
         columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0    business_id      100.00     0.00            0    object
1           name      100.00     0.00            0    object
2        address      100.00     0.00            0    object
3           city      100.00     0.00            0    object
4          state      100.00     0.00            3    object
5    postal_code      100.00     0.00            0    object
6       latitude      100.00     0.00            0    object
7      longitude      100.00     0.00            0    object
8          stars      100.00     0.00            0    object
9   review_count      100.00     0.00            0    object
10       is_open      100.00     0.00            0    object
11    attributes       90.92     9.08        13642    object
12    categories      100.00     0.00            0    object
13         hours       84.61    15.39        23120    object


#### **📤 LOAD**

In [10]:
#guardar en parquet
business.to_parquet("../0_Dataset/Yelp/business.parquet", engine="pyarrow")

### review.json

In [14]:
# Cantidad de datos por porción
chunk_s = 50000

# Ruta del archivo
archivo = '../0_Dataset/Yelp/review.json'

# Inicializar una lista para almacenar las muestras
muestras = []

# Se carga el JSON en un DataFrame por partes y se toma una muestra de cada parte
for df in pd.read_json(archivo, lines=True, chunksize=chunk_s):
    muestra = df.sample(frac=0.03, random_state=1)  # Ajusta el tamaño de la muestra según tus necesidades
    muestras.append(muestra)

# Concatenar todas las muestras en un único DataFrame
df_muestra_completa = pd.concat(muestras, axis=0, ignore_index=True)


In [15]:
data_type_check(df_muestra_completa)
df_muestra_completa.sample(2)


 Resumen del dataframe:

Dimensiones:  (209708, 9)
       columna  %_no_nulos  %_nulos  total_nulos       tipo_dato
0    review_id       100.0      0.0            0          object
1      user_id       100.0      0.0            0          object
2  business_id       100.0      0.0            0          object
3        stars       100.0      0.0            0           int64
4       useful       100.0      0.0            0           int64
5        funny       100.0      0.0            0           int64
6         cool       100.0      0.0            0           int64
7         text       100.0      0.0            0          object
8         date       100.0      0.0            0  datetime64[ns]



📤 LOAD

In [16]:
# Guardar la muestra en un archivo Parquet
muestra_archivo = '../0_Dataset/Yelp/review_reducido.parquet'
df_muestra_completa.to_parquet(muestra_archivo)

### checkin.json

In [9]:
# Ruta del archivo JSON
checkin_path = '../0_Dataset/Yelp/checkin.json'
output_path = '../0_Dataset/Yelp/checkin_reducido.parquet'

# Leer JSON
df = pd.read_json(checkin_path, lines=True)

# Estimar la fracción de datos para que el archivo resultante pese menos de 100 MB
# Aquí asumimos que podemos trabajar con el 10% de los datos originales.
# Ajusta la fracción según sea necesario después de verificar el tamaño del archivo resultante.
fraccion = 0.70

# Tomar una muestra aleatoria del dataset
df_muestra = df.sample(frac=fraccion, random_state=42)


Tamaño del archivo Parquet reducido: 87.89 MB
El archivo es menor de 90 MB. Podrías aumentar la fracción.


In [ ]:
data_type_check(df_muestra)
df_muestra.sample(2)


📤 LOAD

In [ ]:

# Guardar la muestra como un archivo Parquet
pq.write_table(pa.Table.from_pandas(df_muestra), output_path)

# Verificar el tamaño del archivo resultante
file_size = os.path.getsize(output_path) / (1024 * 1024)  # Convertir a MB
print(f'Tamaño del archivo Parquet reducido: {file_size:.2f} MB')

# Ajustar la fracción si es necesario
if file_size > 100:
    print("El archivo sigue siendo demasiado grande. Reduce la fracción y vuelve a intentarlo.")
elif file_size < 90:
    print("El archivo es menor de 90 MB. Podrías aumentar la fracción.")
else:
    print("El tamaño del archivo es adecuado.")

### tip.json

In [20]:
# Ruta del archivo JSON
tip = '../0_Dataset/Yelp/tip.json'

# Leer el archivo JSON
df_tip = pd.read_json(tip, lines=True)


In [ ]:
data_type_check(df_tip)
df_tip.sample(2)

In [ ]:
# Almacenar en formato Parquet, cambiando el sufijo '.json' a '.parquet'
df_tip.to_parquet(tip.replace('.json', '.parquet'))

### user.parquet


In [4]:
# Ruta del archivo Parquet original
user_path = '../0_Dataset/Yelp/user.parquet'

# Leer el archivo Parquet
df = pd.read_parquet(user_path)

# Obtener una muestra aleatoria del 3% del DataFrame (ajustar el tamaño según sea necesario)
sample_df = df.sample(frac=0.03, random_state=42)


In [ ]:
data_type_check(sample_df)
sample_df.sample(2)

In [ ]:

# Ruta del archivo Parquet de muestra
sample_path = '../0_Dataset/Yelp/user_reducido.parquet'

# Guardar la muestra en un nuevo archivo Parquet
sample_df.to_parquet(sample_path, index=False, compression='snappy')

print(f"Muestra guardada en {sample_path}")

___

## Dataset Gogle


 **📂Procesamiento del 1er archivo: `Google Maps/metadata-sitios/review-Florida-`**

### Reviews Florida

#### 📦 **Extraccion** de los datos y primera exploración 
 

🌟 Primero, vamos a convertir los datos de las reseñas de Google Maps del estado de Florida, distribuidos en 11 archivos JSON

In [11]:
# Se especifica la ruta que contiene la carpeta con los archivos:
carpeta = "../0_Dataset/review-Florida"

# Se crea lista vacía donde se almacenarán los dataframes de cada archivo:
reviews = []

# Se recorre por todos los archivos en la carpeta:
for filename in os.listdir(carpeta):
    if filename.endswith('.json'):
        # Se carga el archivo JSON en un DataFrame de Pandas:
        filepath = os.path.join(carpeta, filename)
        df = pd.read_json(filepath, lines = True)
        
        # Se agrega el DataFrame a la lista:
        reviews.append(df)

# Se combinan todos los DataFrames en uno solo usando pd.concat:
df_rev_FL = pd.concat(reviews, ignore_index=True)

,user_id,name,time,rating,text,pics,resp,gmap_id
222749,1.158438e+20,Eliezer Maxime,1610445470638,5,None,None,None,0x88dd717495ae0a6d:0x1f7f551c2c6eccb1
2824985,1.050261e+20,alejandro sanchez,1600115059710,4,None,None,None,0x88d9c04e0e2036eb:0xc3bd5480cbaf8787


In [12]:
data_type_check(df_rev_FL)
df_rev_FL.sample(2)


 Resumen del dataframe:

Dimensiones:  (2850000, 8)
   columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0  user_id      100.00     0.00            0   float64
1     name      100.00     0.00            0    object
2     time      100.00     0.00            0     int64
3   rating      100.00     0.00            0     int64
4     text       62.12    37.88      1079510    object
5     pics        3.66    96.34      2745810    object
6     resp       15.98    84.02      2394601    object
7  gmap_id      100.00     0.00            0    object


#### **📤 LOAD**

In [19]:
df_rev_FL_muestra = df_rev_FL.sample(frac=0.25, random_state=1) 
# Se exporta el archivo en formato parquet:
df_rev_FL_muestra.to_parquet("../0_Dataset/Google/G_review_FL.parquet", engine="pyarrow")

### Metadata-sitios

🌟 La "metadata" incluye la información de diversos establecimientos en Google Maps.

📂 Esta información está dividida en 11 archivos JSON, organizados en 3 carpetas para facilitar el procesamiento y almacenamiento.

In [17]:
# Se especifica la ruta que contiene la carpeta con los archivos:
carpeta = "../0_Dataset/Google/metadata-sitios"

# Se crea lista vacía donde se almacenarán los dataframes de cada archivo:
metadata = []

# Se recorre por todos los archivos en la carpeta:
for filename in os.listdir(carpeta):
    if filename.endswith('.json'):
        # Se carga el archivo JSON en un DataFrame de Pandas:
        filepath = os.path.join(carpeta, filename)
        df1 = pd.read_json(filepath, lines = True)
        
        # Se agrega el DataFrame a la lista:
        metadata.append(df1)

# Se combinan todos los DataFrames en uno solo usando pd.concat:
df_rev_FL = pd.concat(metadata, ignore_index=True)



,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
2142763,"Haymaker - Water Fill Station North Of Dover, OK","Haymaker - Water Fill Station North Of Dover, ...",0x87ae22327e5b4bbb:0x18cd9dc8bac26971,None,36.044407,-97.883274,[Water works],5.0,5,None,"[[Sunday, Open 24 hours], [Monday, Open 24 hou...",None,Open 24 hours,None,https://www.google.com/maps/place//data=!4m2!3...
969100,Salem Sayers Baptist Academy,"Salem Sayers Baptist Academy, 5212 FM1628, Adk...",0x865cefc090aa26e7:0xe2fafaabbd522062,None,29.373403,-98.288462,[Private educational institution],5.0,2,None,"[[Wednesday, 6:30AM–6PM], [Thursday, 6:30AM–6P...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x865cefc0eb8253b3:0xc95ca26a3da6e872, 0x865c...",https://www.google.com/maps/place//data=!4m2!3...


In [18]:
data_type_check(df_rev_FL)
df_rev_FL.sample(2)


 Resumen del dataframe:

Dimensiones:  (2200008, 15)
             columna  %_no_nulos  %_nulos  total_nulos tipo_dato
0               name      100.00     0.00           30    object
1            address       97.09     2.91        64075    object
2            gmap_id      100.00     0.00            0    object
3        description        7.20    92.80      2041504    object
4           latitude      100.00     0.00            0   float64
5          longitude      100.00     0.00            0   float64
6           category       99.37     0.63        13824    object
7         avg_rating      100.00     0.00            0   float64
8     num_of_reviews      100.00     0.00            0     int64
9              price        7.72    92.28      2030122    object
10             hours       72.78    27.22       598803    object
11              MISC       75.59    24.41       536945    object
12             state       74.20    25.80       567655    object
13  relative_results       89.55    

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
414902,Ralph's Barber Shop,"Ralph's Barber Shop, 1827-A, Reynolds Ave, Nor...",0x88fe65e4c7dd302f:0x97afae4dd6103404,None,32.858370,-79.969606,[Barber shop],4.9,28,None,"[[Tuesday, 10AM–6PM], [Wednesday, 10AM–6PM], [...","{'Highlights': ['LGBTQ friendly'], 'Accessibil...",Open ⋅ Closes 6PM,"[0x88fe64e8907df079:0x82bc492c4ace3924, 0x88fe...",https://www.google.com/maps/place//data=!4m2!3...
1174132,Eyeworks At Midtown,"Eyeworks At Midtown, 316 Gray St, Houston, TX ...",0x8640bf41a157d6cd:0xc10fff1ee4d69980,None,29.751944,-95.377058,"[Eye care center, Optometrist]",4.6,8,None,"[[Thursday, 10AM–5PM], [Friday, Closed], [Satu...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 10AM,"[0x8640bf6d03eca757:0x19ac5705011d7bf0, 0x8640...",https://www.google.com/maps/place//data=!4m2!3...


📤 LOAD

In [21]:
df_rev_FL_muestra = df_rev_FL.sample(frac=0.10, random_state=1) 

In [22]:
# Se exporta el archivo en formato parquet:
df_rev_FL_muestra.to_parquet("../0_Dataset/Google/G_metadata_FL_reducido.parquet", engine="pyarrow")
gc.collect()

0

In [ ]:
#  Posibles soluciones: Exportar multiples parquet de metadata y unirlos luego 
 